In [2]:
import torch

In [3]:
# Load the model
model = torch.load('Neuro-GPT/pretrained_model/pytorch_model.bin', map_location=torch.device('cpu'))

In [4]:
model.forward

AttributeError: 'collections.OrderedDict' object has no attribute 'forward'

In [13]:
def analyze_input_layer(state_dict):
    # Get the first convolutional layer weights
    first_layer_weight = state_dict['encoder.patch_embedding.shallownet.0.weight']
    
    # Extract key dimensions
    out_channels, in_channels, kernel_height, kernel_width = first_layer_weight.shape
    
    print(f"First Layer Analysis:")
    print(f"Input Channels: {in_channels}")
    print(f"Output Channels: {out_channels}")
    print(f"Kernel Size: {kernel_height}x{kernel_width}")
    
    return {
        'in_channels': in_channels,
        'out_channels': out_channels,
        'kernel_size': (kernel_height, kernel_width)
    }

# Analyze the model
dims = analyze_input_layer(model)

First Layer Analysis:
Input Channels: 1
Output Channels: 40
Kernel Size: 1x25


In [14]:
analyze_input_layer(model)


First Layer Analysis:
Input Channels: 1
Output Channels: 40
Kernel Size: 1x25


{'in_channels': 1, 'out_channels': 40, 'kernel_size': (1, 25)}